In [1]:
import numpy as np
from numpy import pi
# importing Qiskit
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram, plot_bloch_multivector

In [2]:
def qft_rotations(circuit, n):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(pi/2**(n-qubit), qubit, n)
    # At the end of our function, we call the same function again on
    # the next qubits (we reduced n by one earlier in the function)
    qft_rotations(circuit, n)

def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft(circuit, n):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [3]:
n = 4
qc = QuantumCircuit(n)
qft(qc,n)
# qc.draw()

In [4]:
trans_qc = transpile(
    qc,
    basis_gates=['x', 'h', 'rz', 'add', 'cx'],
)

In [5]:
print(trans_qc.count_ops())
print("total:", sum(trans_qc.count_ops().values()))
print("cx:", trans_qc.count_ops()['cx'])

OrderedDict([('rz', 18), ('cx', 18), ('h', 4)])
total: 40
cx: 18


In [6]:
trans_qc.qasm(filename="./raw_qasm/qft_n{}_from_qiskit.qasm".format(n))

'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[4];\nrz(pi/16) q[0];\nrz(pi/8) q[1];\nrz(pi/4) q[2];\nh q[3];\ncx q[0],q[3];\nrz(-pi/16) q[3];\ncx q[0],q[3];\nrz(pi/8) q[0];\nrz(pi/16) q[3];\ncx q[1],q[3];\nrz(-pi/8) q[3];\ncx q[1],q[3];\nrz(pi/4) q[1];\nrz(pi/8) q[3];\ncx q[2],q[3];\nrz(-pi/4) q[3];\ncx q[2],q[3];\nh q[2];\ncx q[0],q[2];\nrz(-pi/8) q[2];\ncx q[0],q[2];\nrz(pi/4) q[0];\nrz(pi/8) q[2];\ncx q[1],q[2];\nrz(-pi/4) q[2];\ncx q[1],q[2];\nh q[1];\ncx q[0],q[1];\nrz(-pi/4) q[1];\ncx q[0],q[1];\nh q[0];\nrz(pi/4) q[1];\nrz(pi/4) q[2];\ncx q[1],q[2];\ncx q[2],q[1];\ncx q[1],q[2];\nrz(pi/4) q[3];\ncx q[0],q[3];\ncx q[3],q[0];\ncx q[0],q[3];\n'